In [ ]:
import numpy as np
import pandas as pd

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\AA}[]{\unicode{x212B}}$

Calculate coherence decay of an electron spin as a result of interactions with nuclear spin bath for a given supercell. An electron spin is initialized in a superposition state $\ket{0}+\ket{1}$ (or other) then the spin state is evolved under the Master equation through a spin hamiltonian defined by the crystal lattice. Cluster Correlation Expansion method is used to separate the interaction space into clusters to reduce the size of the Hilbert space in diagonalizing the hamiltonian.

In [ ]:
"""declare variable here"""
cce_order=2
mag_field=[0,0,5.0] #Tesla
spin_gyromagratio = 2.00 #M rad s-1 T-1
cluster_cutoff= 5 #Angstrom
final_time = 1e-5 # in s
time_step = 1000 #number of steps
bath_radial_cutoff=50 #Angstrom
electron_zeeman_file=path_to_GIPAW_output
bath_interaction=path_to_GIPAW_output
spin_bath_interaction=path_to_GIPAW_output
geometry_file = path_to_CONTCAR
supercell_repeat= (1,1,3)
spin_position= [0,0,0] #change to sp3 carbon

here we tell the program to set spin at a specific location and tell it 
to prepare an electronic state at $\ket{0}+\ket{1}$ after Hahn's initial 90$^{\circ}$ pulse. State $\ket{0}$ and $\ket{1}$ are separated by Zeeman energy splitting which we obtained from GIPAW.

system hamiltonian is purely Zeeman assuming electron spin resides on a C-12 site (see https://static-content.springer.com/esm/art%3A10.1038%2Fncomms12935/MediaObjects/41467_2016_BFncomms12935_MOESM89_ESM.pdf for hamiltonian):


In [ ]:
#create e_spin object

sp3_defect_spin = spin(gamma_e=spin_gyromagratio,
              e_zeeman=read_from_GIPAW(file_name),
              spin_position=spin_position,
              mag_field)

give bath position as CONTCAR and create a supercell. Apply r_cutoff and  populate the sites randomly with isotopes based on natural abundance. Also initialize the bath state by randomly assign either aligned or anti-aligned with mag_field.

Bath interaction is purely the secular dipolar interactions (see SpinDec manual for formula):
$$\hat{H}_{D} = D\hat{S}^{z}_{1}\hat{S}^{z}_{2} - \frac{D}{4}[\hat{S}^{+}_{1}\hat{S}^{-}_{2}+\hat{S}^{-}_{1}\hat{S}^{+}_{2}],$$

with 

$$D=\frac{D_{0}\gamma_{1}\gamma_{2}(1-3cos^2(\theta))}{r^3}$$

- $\gamma_1,\gamma_2 [M rad s^{-1}T^{-1}]$ are gyromacnetic ratios
- $\theta [rad]$ is the angle between the magnetic field direction and the vector connecting spin1,2
- $r [m]$: distance between two spins
- $D_{0} = 10^6 (\frac{\mu_0\hbar{}}{4\pi}) [M rad s^{-1} m^{3}]$
- $\hbar [Js]$

In [ ]:
#create spin_bath object
SWCNT_Ph_NO2_spin = spin_bath(bath_positions = path_to_CONTCAR,
                              repeat = supercell_repeat,
                              r_cutoff = bath_radial_cutoff,
                              interaction = bath_interaction, 
                              mag_field)

pair system(e spin) and bath together and gives system-bath interactions given by GIPAW

In [ ]:
system_bath = (system=sp3_defect_spin, 
               bath = SWCNT_Ph_NO2_spin, 
               interaction=spin_bath_interaction, field = mag_field)

Create a vertices which stores interacting clusters based on cluster cutoff, cce order. For example, order=2, electron site may interact with a bath site that is 10 $\AA{}$ away but that site also includes interactions with one site that is within 5 $\AA{}$. We need to include all such pairs to properly define spin hamiltonian in truncated space.

In [ ]:
bath_vertices = CCE_vertices(bath_sites=SWCNT_Ph_NO2_spin.get_bath_geometry(),
                             order=cce_order,
                             cluster_cutoff = cluster_cutoff)

define time array and solve CCE problem. Here we solve the master equation with spin Hamiltonian and calculate decoherence with the bath density matrix.

In [ ]:
time_array = np.linspace(0, final_time, time_step, endpoint=True)
cce = CCE(system_bath, 
          cce_order, 
          bath_vertices, 
          time_array)
cce.calculate()

In [ ]:
cce.plot_decoherence()

In [ ]:
T2=cce.fit_T2()
print("T2 = "+T2+" ms")